In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
#load the supporting airports & carriers data set 
df_airports = pd.read_csv('data files/airports.csv')
df_carriers = pd.read_csv('data files/carriers.csv')

In [3]:
#load the main flights data set
df = pd.read_csv('data files/2008.csv')

In [4]:
#view df_airports
df_airports.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


In [5]:
# add Dest & (Dest & Origin) cities, airports & states  columns containing iata data and rename iata to Origin for joining tables 
df_airports['Dest'] = df_airports.iata
df_airports.rename(columns= {'iata': 'Origin'},inplace =True)
df_airports['Dest_city'] = df_airports.city
df_airports['Dest_airport'] = df_airports.airport
df_airports['Dest_state'] = df_airports.state
df_airports['Origin_city'] = df_airports.city
df_airports['Origin_airport'] = df_airports.airport
df_airports['Origin_state'] = df_airports.state
df_airports.drop(columns=['city','state', 'airport'],inplace =True)
#view df_airports after changes
df_airports.head()

,Origin,country,lat,long,Dest,Dest_city,Dest_airport,Dest_state,Origin_city,Origin_airport,Origin_state
0,00M,USA,31.953765,-89.234505,00M,Bay Springs,Thigpen,MS,Bay Springs,Thigpen,MS
1,00R,USA,30.685861,-95.017928,00R,Livingston,Livingston Municipal,TX,Livingston,Livingston Municipal,TX
2,00V,USA,38.945749,-104.569893,00V,Colorado Springs,Meadow Lake,CO,Colorado Springs,Meadow Lake,CO
3,01G,USA,42.741347,-78.052081,01G,Perry,Perry-Warsaw,NY,Perry,Perry-Warsaw,NY
4,01J,USA,30.688012,-81.905944,01J,Hilliard,Hilliard Airpark,FL,Hilliard,Hilliard Airpark,FL


In [6]:
#view df_carriers
df_carriers.head()

,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [7]:
#  rename Code to UniqueCarrier and  Description to CarrierName for joining tables
df_carriers = df_carriers.rename(columns= {'Code': 'UniqueCarrier' , 'Description': 'CarrierName'})
#view df_airports after changes
df_carriers.head()


,UniqueCarrier,CarrierName
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [8]:
print("check fight data data types before cleaning")
df.info()



check fight data data types before cleaning
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(

In [9]:
# convert CRSArrTime, CRSDepTime, ArrTime & DepTime times to time string & gettig Dep & Arr hours

def times_to_time_str(row):
    
    if not math.isnan(row['DepTime']) :
        DepTime_str = str(row['DepTime']).split(".")[0]
        DepTime_len = len(DepTime_str)
        if DepTime_len < 4 :
            if DepTime_len == 3:
                DepTime.append('0{}:{}'.format(DepTime_str[:1],DepTime_str[1:])+":00")
            elif DepTime_len == 2:
                DepTime.append("00:{}".format(DepTime_str)+":00")
            else: 
                DepTime.append("00:0{}".format(DepTime_str)+":00")
        else:
            DepTime.append('{}:{}'.format(DepTime_str[:2],DepTime_str[2:])+":00")
    else:
        DepTime.append(np.nan)
        
        
    if not math.isnan(row['ArrTime']) :
        ArrTime_str = str(row['ArrTime']).split(".")[0]
        ArrTime_len = len(ArrTime_str)
        if ArrTime_len < 4 :
            if ArrTime_len == 3:
                ArrTime.append('0{}:{}'.format(ArrTime_str[:1],ArrTime_str[1:])+":00")
            elif ArrTime_len == 2:
                ArrTime.append("00:{}".format(ArrTime_str)+":00")
            else: 
                ArrTime.append("00:0{}".format(ArrTime_str)+":00")
        else:
            ArrTime.append('{}:{}'.format(ArrTime_str[:2],ArrTime_str[2:])+":00")
    else:
        ArrTime.append(np.nan)
    
    
    CRSDepTime_len = len(str(row['CRSDepTime']))
    if CRSDepTime_len < 4 :
        if CRSDepTime_len == 3:
            CRSDepTime.append('0{}:{}'.format(str(row['CRSDepTime'])[:1],str(row['CRSDepTime'])[1:])+":00")
            DepHour.append(str(row['CRSDepTime'])[:1])
        elif CRSDepTime_len == 2:
            CRSDepTime.append("00:{}".format(row['CRSDepTime'])+":00")
            DepHour.append(str(0))
        else: 
            CRSDepTime.append("00:0{}".format(row['CRSDepTime'])+":00")
            DepHour.append(str(0))
    else:
        CRSDepTime.append('{}:{}'.format(str(row['CRSDepTime'])[:2],str(row['CRSDepTime'])[2:])+":00")
        DepHour.append(str(row['CRSDepTime'])[:2])
    
    CRSArrTime_len = len(str(row['CRSArrTime']))
    if CRSArrTime_len < 4 :
        if CRSArrTime_len == 3:
            CRSArrTime.append('0{}:{}'.format(str(row['CRSArrTime'])[:1],str(row['CRSArrTime'])[1:])+":00")
            ArrHour.append(str(row['CRSArrTime'])[:1])
        elif CRSArrTime_len == 2:
            CRSArrTime.append("00:{}".format(row['CRSArrTime'])+":00")
            ArrHour.append(str(0))
        else: 
            CRSArrTime.append("00:0{}".format(row['CRSArrTime'])+":00")
            ArrHour.append(str(0))
    else:
        CRSArrTime.append('{}:{}'.format(str(row['CRSArrTime'])[:2],str(row['CRSArrTime'])[2:])+":00")
        ArrHour.append(str(row['CRSArrTime'])[:2])



In [10]:

CRSDepTime = []
CRSArrTime = []
DepTime = []
ArrTime = []
DepHour = []
ArrHour = []
print("1- dropping 'TailNum','FlightNum','Diverted', 'ActualElapsedTime','CRSElapsedTime','TaxiIn' & 'TaxiOut' colunms")
df = df.drop(columns=['TailNum', 'ActualElapsedTime','CRSElapsedTime','TaxiIn','TaxiOut', 'FlightNum','Diverted'])
print('2- convert CRSArrTime, CRSDepTime, ArrTime & DepTime times to time string and gettig Dep & Arr hours') 
df.apply(times_to_time_str, axis=1)
df['DepHour'] = DepHour
df['ArrHour'] = ArrHour
print('3- convert CRSArrTime, CRSDepTime, ArrTime & DepTime string to timedelta')
df['ArrTime'] = pd.to_timedelta(ArrTime,errors = 'coerce')
df['DepTime'] = pd.to_timedelta(DepTime, errors = 'coerce')
df['CRSArrTime'] = pd.to_timedelta(CRSArrTime)
df['CRSDepTime'] = pd.to_timedelta(CRSDepTime)
print('4- get the Origin & Destination airports, citys and states form airports data')
df = pd.merge(df,df_airports[['Dest','Dest_airport','Dest_city','Dest_state']],on='Dest', how='left')
df = pd.merge(df,df_airports[['Origin','Origin_airport','Origin_city','Origin_state']],on='Origin', how='left')
print('5- get the flight carrier data ')
df = pd.merge(df,df_carriers[['UniqueCarrier','CarrierName']],on='UniqueCarrier', how='left')


1- dropping 'TailNum','FlightNum','Diverted', 'ActualElapsedTime','CRSElapsedTime','TaxiIn' & 'TaxiOut' colunms
2- convert CRSArrTime, CRSDepTime, ArrTime & DepTime times to time string and gettig Dep & Arr hours
3- convert CRSArrTime, CRSDepTime, ArrTime & DepTime string to timedelta
4- get the Origin & Destination airports, citys and states form airports data
5- get the flight carrier data 


In [13]:
print("check fight data data types after cleaning")
df.info()


check fight data data types after cleaning
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7009728 entries, 0 to 7009727
Data columns (total 31 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              timedelta64[ns]
CRSDepTime           timedelta64[ns]
ArrTime              timedelta64[ns]
CRSArrTime           timedelta64[ns]
UniqueCarrier        object
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
Cancelled            int64
CancellationCode     object
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
DepHour              object
ArrHour              object
Dest_airport         object
Dest_city            object
Dest_state           object
Origin_airport       object
Origin_city          o

In [14]:
print('6- saving cleaned filghts data as 2008_cleaned.csv in data files folder ')
df.to_csv('data files/2008_cleaned.csv')


6- saving cleaned filghts data as 2008_cleaned.csv in data files folder 
